In [299]:
import pandas as pd
import plotly.express as px

df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [300]:
df = df.reset_index(drop= True)

### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

#### User-Vergangenheit

##### prepo

In [302]:
df_userhh = df.copy()

In [303]:
df_userhh = df_userhh.sort_values(by= ["user_id", "order_number"], ascending= True)

In [304]:
df_userh1 = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "unique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"), order_number= ("order_number", "unique"))

In [305]:
df_userh1

,user_id,orders,reordered,tip,order_number
0,3,"[244029, 78693, 533119, 361761, 481466, 350475...",0.625000,0.840909,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,7,"[455863, 426645, 21397, 301146, 589317, 298684...",0.679070,0.637209,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,14,"[515987, 569030, 242632, 245955, 300451, 37770...",0.343891,0.506787,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
3,23,"[318279, 1415, 337923, 190237, 603547]",0.192308,0.000000,"[1, 2, 3, 4, 5]"
4,25,"[463866, 531613, 45667]",0.095238,0.476190,"[1, 2, 3]"
...,...,...,...,...,...
37362,206186,"[325279, 330436, 373011, 28440]",0.307692,0.564103,"[1, 2, 3, 4]"
37363,206189,"[526741, 350559, 213452, 317551, 265207, 345993]",0.475000,0.250000,"[1, 2, 3, 4, 5, 6]"
37364,206192,"[575407, 161466, 525470, 24105, 16470, 447330,...",0.640777,0.223301,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
37365,206195,"[164347, 403007, 557442, 433213, 398328, 14601...",0.438356,0.136986,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [306]:
last_orders = []

for ind, order in enumerate(df_userh1.orders):
 last_orders.append(df_userh1.orders[ind][-1])

In [307]:
df_userhh.drop(df_userhh.loc[df_userhh.order_id.isin(last_orders)].index, inplace= True)

In [308]:
df_userh1.orders[0][-1]

248921

In [309]:
df_userh = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "nunique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))

In [310]:
df_userh

,user_id,orders,reordered,tip
0,3,11,0.597561,0.829268
1,7,20,0.669903,0.621359
2,14,13,0.323810,0.533333
3,23,4,0.100000,0.000000
4,25,2,0.066667,0.266667
...,...,...,...,...
37362,206186,3,0.281250,0.687500
37363,206189,5,0.457143,0.142857
37364,206192,13,0.618557,0.237113
37365,206195,19,0.432836,0.149254


In [311]:
df_last_order = df.loc[df.order_id.isin(last_orders)].groupby("user_id").agg(tip_last_order= ("tip", "max"), last_order_id = ("order_id", "max")).reset_index()

In [312]:
df_last_order

,user_id,tip_last_order,last_order_id
0,3,1,248921
1,7,1,93090
2,14,0,411382
3,23,0,603547
4,25,1,45667
...,...,...,...
37362,206186,0,28440
37363,206189,1,345993
37364,206192,0,111678
37365,206195,0,530219


In [313]:
df_userh = df_userh.merge(df_last_order, how= "right", on= "user_id")

In [314]:
df_userh

,user_id,orders,reordered,tip,tip_last_order,last_order_id
0,3,11,0.597561,0.829268,1,248921
1,7,20,0.669903,0.621359,1,93090
2,14,13,0.323810,0.533333,0,411382
3,23,4,0.100000,0.000000,0,603547
4,25,2,0.066667,0.266667,1,45667
...,...,...,...,...,...,...
37362,206186,3,0.281250,0.687500,0,28440
37363,206189,5,0.457143,0.142857,1,345993
37364,206192,13,0.618557,0.237113,0,111678
37365,206195,19,0.432836,0.149254,0,530219


##### Analyse

**Bestellungen bei welcher der user eine tip_wshk von >= 0.5 hat verglichen mit seiner letzten/aktuellen Bestellung**


Unter der Annahme, dass wenn die tip_wshk eines users in der historie >= 0.5 ist, tendiert er eher dazu zu tippen bei seiner letzten/aktuellen Bestellung

**trifft zu:**

In [315]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 1)]

,user_id,orders,reordered,tip,tip_last_order,last_order_id
0,3,11,0.597561,0.829268,1,248921
1,7,20,0.669903,0.621359,1,93090
6,39,6,0.571429,0.500000,1,489351
13,73,6,0.363636,0.636364,1,443957
15,85,6,0.384615,0.807692,1,431764
...,...,...,...,...,...,...
37334,206038,7,0.319444,1.000000,1,265052
37348,206130,16,0.705426,0.844961,1,403088
37351,206140,9,0.466165,0.571429,1,8611
37354,206153,13,0.456000,0.536000,1,113860


In [316]:
7685 / 37367

0.2056627505553028

**trifft nicht zu:**

In [317]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 0)] #! Hier schauen ob die Attribute die Bestellung beeinflussen

,user_id,orders,reordered,tip,tip_last_order,last_order_id
2,14,13,0.323810,0.533333,0,411382
20,110,20,0.556196,0.700288,0,12722
26,138,32,0.628378,0.533784,0,80767
27,140,76,0.735882,0.537793,0,454907
33,169,6,0.105882,0.917647,0,136338
...,...,...,...,...,...,...
37333,206030,37,0.702550,0.526912,0,340421
37342,206086,46,0.825131,0.596859,0,296139
37344,206093,17,0.625000,0.776786,0,105730
37346,206116,5,0.448276,0.568966,0,319350


In [318]:
4181 / 37367

0.11189017047127145

**Bestellungen bei welcher der user eine tip_wshk von < 0.5 hat verglichen mit seiner letzten/aktuellen Bestellung**

Unter der Annahme, dass wenn die tip_wshk eines users in der historie < 0.5 ist, tendiert er eher dazu nicht zu tippen bei seiner letzten/aktuellen Bestellung

**trifft zu:**

In [319]:
df_userh.loc[(df_userh.tip < 0.5) & (df_userh.tip_last_order == 0)]

,user_id,orders,reordered,tip,tip_last_order,last_order_id
3,23,4,0.100000,0.000000,0,603547
9,53,3,0.028571,0.000000,0,548094
10,55,7,0.505747,0.000000,0,6945
11,65,14,0.393939,0.000000,0,190242
14,83,6,0.333333,0.179487,0,430576
...,...,...,...,...,...,...
37359,206168,11,0.360825,0.371134,0,108087
37361,206185,10,0.466667,0.000000,0,363750
37364,206192,13,0.618557,0.237113,0,111678
37365,206195,19,0.432836,0.149254,0,530219


In [320]:
20635 / 37367

0.5522252254663206

**trifft nicht zu:**

In [321]:
df_userh.loc[(df_userh.tip < 0.5) & (df_userh.tip_last_order == 1)] #! Hier nach den Items im Warenkorb bei den aktuellen Bestellungen schauen

,user_id,orders,reordered,tip,tip_last_order,last_order_id
4,25,2,0.066667,0.266667,1,45667
5,28,23,0.416667,0.303571,1,351338
7,40,8,0.641304,0.456522,1,203217
8,42,16,0.548872,0.127820,1,497941
12,72,12,0.680851,0.212766,1,463486
...,...,...,...,...,...,...
37335,206045,32,0.551807,0.407229,1,566792
37338,206068,17,0.461538,0.179487,1,179583
37343,206089,8,0.750000,0.125000,1,73987
37358,206162,18,0.453608,0.479381,1,23102


In [322]:
4866 / 37367

0.1302218535071052

#### Inhalt und Parameter der aktuellen Bestellungen

**Warenkörbe der Bestellungen die bei der Hypothese nicht zutreffen:**

In [345]:
df_waren = df_userh.loc[((df_userh.tip >= 0.5) & (df_userh.tip_last_order == 0)) | ((df_userh.tip < 0.5) & (df_userh.tip_last_order == 1))][["user_id", "last_order_id"]]
df_waren.head()

,user_id,last_order_id
2,14,411382
4,25,45667
5,28,351338
7,40,203217
8,42,497941


In [348]:
df.loc[df.user_id.isin(df_waren.user_id)].sort_values(by=["order_number"], ascending= True).groupby(["user_id","order_id"], as_index= False).agg(
 tip =("tip", "mean"),
 days_since_prior_order= ("days_since_prior_order", "max"),
 product_name= ("product_name", "nunique"),
 order_hour_of_day = ("order_hour_of_day", "max"),
 order_dow = ("order_dow", "max"),
 reordered = ("reordered", "mean")
 )

,user_id,order_id,tip,days_since_prior_order,product_name,order_hour_of_day,order_dow,reordered
0,14,237041,1.0,25.0,13,5,1,0.384615
1,14,242632,1.0,2.0,3,14,0,0.666667
2,14,245955,0.0,20.0,27,8,6,0.148148
3,14,288644,0.0,30.0,9,21,4,0.333333
4,14,300451,1.0,30.0,23,8,5,0.173913
...,...,...,...,...,...,...,...,...
161248,206189,265207,0.0,22.0,6,15,3,0.666667
161249,206189,317551,0.0,30.0,9,13,2,0.777778
161250,206189,345993,1.0,4.0,5,22,0,0.600000
161251,206189,350559,0.0,15.0,5,11,5,0.400000


#### old

In [325]:
df_userh.loc[df_userh.orders < df_userh.orders]

,user_id,orders,reordered,tip,tip_last_order,last_order_id


In [326]:
df_userh.loc[df_userh.orders >= df_userh.orders.quantile(0.8)]

,user_id,orders,reordered,tip,tip_last_order,last_order_id
5,28,23,0.416667,0.303571,1,351338
22,118,27,0.703030,0.721212,1,221600
26,138,32,0.628378,0.533784,0,80767
27,140,76,0.735882,0.537793,0,454907
35,175,23,0.284615,0.000000,0,267244
...,...,...,...,...,...,...
37333,206030,37,0.702550,0.526912,0,340421
37335,206045,32,0.551807,0.407229,1,566792
37342,206086,46,0.825131,0.596859,0,296139
37347,206118,23,0.644068,0.000000,0,598274


In [327]:
list_bins = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df_userh["orders_cut"] = pd.cut(df_userh["orders"], bins= list_bins, include_lowest= True)

In [328]:
df_userh

,user_id,orders,reordered,tip,tip_last_order,last_order_id,orders_cut
0,3,11,0.597561,0.829268,1,248921,"(10.0, 20.0]"
1,7,20,0.669903,0.621359,1,93090,"(10.0, 20.0]"
2,14,13,0.323810,0.533333,0,411382,"(10.0, 20.0]"
3,23,4,0.100000,0.000000,0,603547,"(0.999, 10.0]"
4,25,2,0.066667,0.266667,1,45667,"(0.999, 10.0]"
...,...,...,...,...,...,...,...
37362,206186,3,0.281250,0.687500,0,28440,"(0.999, 10.0]"
37363,206189,5,0.457143,0.142857,1,345993,"(0.999, 10.0]"
37364,206192,13,0.618557,0.237113,0,111678,"(10.0, 20.0]"
37365,206195,19,0.432836,0.149254,0,530219,"(10.0, 20.0]"


In [329]:
df_userh1 = df_userh.groupby("orders_cut", as_index= False).agg(orders= ("orders", "sum"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))
df_userh1

,orders_cut,orders,reordered,tip
0,"(0.999, 10.0]",112339,0.298360,0.273098
1,"(10.0, 20.0]",119855,0.502436,0.369598
2,"(20.0, 30.0]",89244,0.600599,0.464372
3,"(30.0, 40.0]",67886,0.653653,0.520055
4,"(40.0, 50.0]",52614,0.693452,0.547952
5,"(50.0, 60.0]",38822,0.714454,0.532735
6,"(60.0, 70.0]",25258,0.712583,0.517715
7,"(70.0, 80.0]",18421,0.735522,0.462543
8,"(80.0, 90.0]",11926,0.765044,0.477205
9,"(90.0, 100.0]",33183,0.759181,0.410179


In [330]:
df_userh1["orders_cut"] = df_userh1["orders_cut"].astype("str")

In [331]:
px.bar(df_userh1, x= "orders_cut", y= "tip")